In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pyunpack import Archive
import pandas as pd
import numpy as np
import json
import math
import os

In [2]:
page = urlopen('https://github.com/sealneaward/nba-movement-data/tree/master/data').read()
soup = BeautifulSoup(page, "lxml")

curryList = []
for anchor in soup.findAll('a', class_="js-navigation-open" ):
    if 'GSW' in anchor.text and len(anchor.text)==24:
        curryList.append(anchor.text)

In [3]:
def download_extract_data(trackingId):
    datalink = ("https://raw.githubusercontent.com/sealneaward/nba-movement-data/"+
                    "master/data/"+trackingId)
    urllib.request.urlretrieve(datalink, 'E:/Baiduyun Download/chuichan/17 Spring/Scala/finalProject/curry/{trackingId}'.format(trackingId=trackingId))
    Archive('E:/Baiduyun Download/chuichan/17 Spring/Scala/finalProject/curry/{trackingId}'.format(trackingId=trackingId)).extractall('E:/Baiduyun Download/chuichan/17 Spring/Scala/finalProject/curryZipped')
    os.remove('E:/Baiduyun Download/chuichan/17 Spring/Scala/finalProject/curry/{trackingId}'.format(trackingId=trackingId))

In [ ]:
for x in curryList:
    download_extract_data(x)

In [49]:
def processShotData(rawDataDirectory, curryShotData):
    gameId = []
    for file in os.listdir(rawDataDirectory):
        gameId.append(file[:-5])
    gameId = list(map(int, gameId))
    curryShotData = curryShotData.loc[curryShotData['game_id'].isin(gameId)]
    return curryShotData

In [50]:
def flattenMovement(d):    
    player_moments = []
    for m in range(len(d["events"])):
        moments = d["events"][m]["moments"]
        for moment in moments:
            for player in moment[5]:
                player.extend((moments.index(moment), moment[0], moment[1], moment[2], moment[3]))
                player_moments.append(player)
    headers = ["team_id", "player_id", "x_loc", "y_loc", "radius", "moment", "period", "unix_time", "game_clock", "shot_clock"]
    player_moments_df = pd.DataFrame(player_moments, columns=headers)
    return player_moments_df

In [73]:
def combine(d, player_moments_df, curryShotData, gameId):
    curryShotData_temp = curryShotData.loc[curryShotData['game_id'] == (gameId)]
    curryShotData_temp.index = range(0,len(curryShotData_temp))
    
    home = d["events"][0]["home"]
    visitor = d["events"][0]["visitor"]
    home_teamid = home["teamid"]
    visitor_teamid = visitor["teamid"]
    homePlayer_id = []
    for m in range(len(d["events"][0]["home"]["players"])):
        homePlayer_id.append(d["events"][0]["home"]["players"][m]["playerid"])
    visiterPlayer_id = []
    for m in range(len(d["events"][0]["visitor"]["players"])):
        visiterPlayer_id.append(d["events"][0]["visitor"]["players"][m]["playerid"])

    for m in range(len(curryShotData_temp)):
        if curryShotData_temp.iloc[m]["player_id"] in homePlayer_id:
            curryShotData_temp.set_value(m, 'homeTeam', 1)
        elif curryShotData_temp.iloc[m]["player_id"] in visiterPlayer_id:
            curryShotData_temp.set_value(m, 'homeTeam', 0)
        else:
            curryShotData_temp.set_value(m, 'homeTeam', 9)

    #player_moments_df.set_value(0, 'homeTeam', 1)
    for m in range(len(player_moments_df)):
        if player_moments_df.ix[m,"team_id"] == home_teamid:
            player_moments_df.set_value(m, 'homeTeam', 1)
        elif player_moments_df.ix[m,"team_id"] == visitor_teamid:
            player_moments_df.set_value(m, 'homeTeam', 0)
        else:
            player_moments_df.set_value(m, 'homeTeam', 9)

    curryShotData_temp.index = range(0,len(curryShotData_temp))
    player_moments_df.index = range(0,len(player_moments_df))

    for m in range(len(curryShotData_temp)):
        period = curryShotData_temp.iloc[m]["period"]
        minutes_remaining = curryShotData_temp.iloc[m]["minutes_remaining"]
        seconds_remaining = curryShotData_temp.iloc[m]["seconds_remaining"]
        game_clock = 60*minutes_remaining + seconds_remaining


        playersMovement = player_moments_df.loc[player_moments_df['period'] == period]
        playersMovement.index = range(0,len(playersMovement))
        playersMovement = playersMovement.loc[playersMovement['homeTeam'] == (1 - curryShotData_temp.iloc[m]["homeTeam"])]
        playersMovement.index = range(0,len(playersMovement))

        try:    
            closest_game_clock = playersMovement.ix[(playersMovement.game_clock-game_clock).abs().argsort()][:1]
            #closest_game_clock.index = range(0,len(game_clock))
            closest_game_clock = closest_game_clock.iloc[0]['game_clock']
            playersMovement = playersMovement.loc[playersMovement['game_clock'] == closest_game_clock]
            playersMovement.index = range(0,len(playersMovement))


            shotPlayer = player_moments_df.loc[player_moments_df['player_id'] == curryShotData_temp.iloc[m]["player_id"]]
            shotPlayer.index = range(0,len(shotPlayer))
            shotPlayer = shotPlayer.loc[shotPlayer['period'] == period]
            shotPlayer.index = range(0,len(shotPlayer))
            shotPlayer = shotPlayer.loc[shotPlayer['game_clock'] == closest_game_clock][:1]
            #shotPlayer.index = range(0,len(shotPlayer))
            shotPlayer_loc_x = shotPlayer.iloc[0]["x_loc"]
            shotPlayer_loc_y = shotPlayer.iloc[0]["y_loc"]

            for n in range(len(playersMovement)):
                defender_loc_x = playersMovement.iloc[n]["x_loc"]
                defender_loc_y = playersMovement.iloc[n]["y_loc"]
                distance = math.sqrt((shotPlayer_loc_x-defender_loc_x)**2 + (shotPlayer_loc_y-defender_loc_y)**2)
                playersMovement.set_value(n, 'distance', distance)

            minimum_distance = np.nanmin(playersMovement.distance.values)

            defender_information = playersMovement.loc[playersMovement['distance'] == minimum_distance][:1]
            #defender_information.index = range(0,len(defender_information))

            curryShotData_temp.set_value(m, 'defender_id', defender_information.iloc[0]['player_id'])
            for x in range(len(home["players"])):
                if defender_information.iloc[0]['player_id'] == home["players"][x]['playerid']:
                    defender_name = home["players"][x]['firstname'] + " " + home["players"][x]['lastname']
                    curryShotData_temp.set_value(m, 'defender_name', defender_name)
            for y in range(len(visitor["players"])):
                if defender_information.iloc[0]['player_id'] == visitor["players"][y]['playerid']:
                    defender_name = visitor["players"][y]['firstname'] + " " + visitor["players"][y]['lastname']
                    curryShotData_temp.set_value(m, 'defender_name', defender_name)

            curryShotData_temp.set_value(m, 'defender_x_loc', defender_information.iloc[0]['x_loc'])
            curryShotData_temp.set_value(m, 'defender_y_loc', defender_information.iloc[0]['y_loc'])
            curryShotData_temp.set_value(m, 'defender_distance', minimum_distance)
            curryShotData_temp.set_value(m, 'loc_x', shotPlayer_loc_x)
            curryShotData_temp.set_value(m, 'loc_y', shotPlayer_loc_y)
        except:
            continue
    
    return curryShotData_temp

In [74]:
def Main(rawDataDirectory,shotDataDirectory):
    os.chdir(rawDataDirectory)
    fileName = os.listdir(rawDataDirectory)
    curryShotData = pd.read_csv(shotDataDirectory)
    curryShotData = processShotData(rawDataDirectory, curryShotData)
    finalData = pd.DataFrame()
    
    for x in fileName:
        with open("./"+x) as json_data:
            d = json.load(json_data)        
        gameId = int(x[:-5])
        player_moments_df = flattenMovement(d)
        curryShotData_temp = combine(d, player_moments_df, curryShotData, gameId)
        finalData = finalData.append(curryShotData_temp).copy()
    finalData.index = range(0,len(finalData))
    return finalData

In [76]:
directory = "E:/Baiduyun Download/chuichan/17 Spring/Scala/finalProject/curryZipped"
shotDataDirectory = "E:/Baiduyun Download/chuichan/17 Spring/Scala/finalProject/201939.csv"
finalData = Main(directory,shotDataDirectory)
finalData.to_csv("E:/Baiduyun Download/chuichan/17 Spring/Scala/finalProject/CurryFinalData.csv",sep=",")
finalData.tail()

,SCOREMARGIN,Unnamed: 0,action_type,defender_distance,defender_id,defender_name,defender_x_loc,defender_y_loc,event_type,game_date,...,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,time,vtm
773,11,814,Cutting Layup Shot,2.526197,101145.0,Monta Ellis,11.57096,16.25912,Made Shot,20160122,...,2,1,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612744,Golden State Warriors,0021500652:4:6:32,IND
774,9,815,Floating Jump shot,22.092492,1626167.0,Myles Turner,15.96334,25.50013,Made Shot,20160122,...,13,1,2PT Field Goal,Center(C),In The Paint (Non-RA),8-16 ft.,1610612744,Golden State Warriors,0021500652:4:5:31,IND
775,13,816,Jump Shot,5.136170,1626202.0,Joe Young,17.13193,4.58316,Made Shot,20160122,...,25,1,3PT Field Goal,Right Side Center(RC),Above the Break 3,24+ ft.,1610612744,Golden State Warriors,0021500652:4:4:16,IND
776,14,817,Jump Shot,13.902731,101139.0,CJ Miles,13.53673,35.88552,Made Shot,20160122,...,23,1,3PT Field Goal,Left Side(L),Left Corner 3,24+ ft.,1610612744,Golden State Warriors,0021500652:4:3:26,IND
777,16,818,Driving Layup Shot,15.579463,101139.0,CJ Miles,9.58349,25.42482,Made Shot,20160122,...,1,1,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612744,Golden State Warriors,0021500652:4:2:26,IND


In [60]:
df = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df1 = pd.DataFrame()

df1 = df1.append(df)
df1

,A,B
0,1,2
1,3,4
